<br>
<br>
<br>


<style>
    body {
        text-align: center;
    }

    #board {
        /* background-color: skyblue; */
        background-image: url("{{site.baseurl}}/flappybirdsrc/flappybirdbg.png")
    }

</style>

<!-- Audio -->
<audio id="pointSound" src="{{site.baseurl}}/audio/points2.wav" preload="auto"></audio>
<audio id="lostSound" src="{{site.baseurl}}/audio/game-over.wav" preload="auto"></audio>
<audio id="winnerSound" src="{{site.baseurl}}/audio/winner.wav" preload="auto"></audio>

<script>
    //disable arrow key scrolling
    window.addEventListener("keydown", function(e) { if(["Space","ArrowUp","ArrowDown","ArrowLeft","ArrowRight"].indexOf(e.code) > -1) { e.preventDefault(); } }, false);
    
    //Sound when jump 
    function playPointSound() {
    const pointSound = document.getElementById("pointSound");
    pointSound.play();
    }

    //Sound when game ends
    function playLostSound() {
    const lostSound = document.getElementById("lostSound");
    lostSound.play();
    }

    //Sound for score 20
    function playWinnerSound() {
    const winnerSound = document.getElementById("winnerSound");
    winnerSound.play();
    }




    //board
    let board;
    let boardWidth = 360;
    let boardHeight = 640;
    let context;

    //bird
    let birdWidth = 34; //width/height ratio = 408/228 = 17/12
    let birdHeight = 24;
    let birdX = boardWidth/8;
    let birdY = boardHeight/2;
    let birdImg;

    let bird = {
        x : birdX,
        y : birdY,
        width : birdWidth,
        height : birdHeight
    }

    //pipes
    let pipeArray = [];
    let pipeWidth = 64; //width/height ratio = 384/3072 = 1/8
    let pipeHeight = 512;
    let pipeX = boardWidth;
    let pipeY = 0;

    let topPipeImg;
    let bottomPipeImg;

    //physics
    let velocityX = -2; //pipes moving left speed
    let velocityY = 0; //bird jump speed
    let gravity = 0.4;

    let gameOver = false;
    let score = 0;

    window.onload = function() {
        board = document.getElementById("board");
        board.height = boardHeight;
        board.width = boardWidth;
        context = board.getContext("2d"); //used for drawing on the board


        //load images
        birdImg = new Image();
        birdImg.src = "{{site.baseurl}}/flappybirdsrc/flappybird.png"; 
        birdImg.onload = function() {
            context.drawImage(birdImg, bird.x, bird.y, bird.width, bird.height);
        }

        topPipeImg = new Image();
        topPipeImg.src = "{{site.baseurl}}/flappybirdsrc/toppipe.png"; //toppipe

        bottomPipeImg = new Image();
        bottomPipeImg.src = "{{site.baseurl}}/flappybirdsrc/bottompipe.png"; //bottompipe

        requestAnimationFrame(update);
        setInterval(placePipes, 1500); //every 1.5 seconds
        document.addEventListener("keydown", moveBird);
    }

    function update() {
        requestAnimationFrame(update);
        if (gameOver) {
            playLostSound();
            return;
        }
        context.clearRect(0, 0, board.width, board.height);

        //bird
        velocityY += gravity;
        bird.y = Math.max(bird.y + velocityY, 0); //apply gravity to current bird.y, limit the bird.y to top of the canvas
        context.drawImage(birdImg, bird.x, bird.y, bird.width, bird.height);

        if (bird.y > board.height) {
            gameOver = true;
        }

        //pipes
        for (let i = 0; i < pipeArray.length; i++) {
            let pipe = pipeArray[i];
           
            var scorediff = score;
            pipe.x += velocityX;
            pipe.x -= 2;
            

            context.drawImage(pipe.img, pipe.x, pipe.y, pipe.width, pipe.height);

            if (!pipe.passed && bird.x > pipe.x + pipe.width) {
                score += 0.5; //0.5 because there are 2 pipes! so 0.5*2 = 1, 1 for each set of pipes
                playPointSound();
                pipe.passed = true;
            }

            if (detectCollision(bird, pipe)) {
                gameOver = true;
            }
        }

        //clear pipes
        while (pipeArray.length > 0 && pipeArray[0].x < -pipeWidth) {
            pipeArray.shift(); //removes first element from the array
        }

        //score
        context.fillStyle = "red";
        context.font="45px Open Sans";
        context.fillText(score, 5, 45);

        if (gameOver) {
            context.fillText("GAME OVER", 5, 90);
        }
    }

    function placePipes() {
        if (gameOver) {
            return;
        }

        //(0-1) * pipeHeight/2.
        // 0 -> -128 (pipeHeight/4)
        // 1 -> -128 - 256 (pipeHeight/4 - pipeHeight/2) = -3/4 pipeHeight
        let randomPipeY = pipeY - pipeHeight/4 - Math.random()*(pipeHeight/2);
        let openingSpace = board.height/4;

        let topPipe = {
            img : topPipeImg,
            x : pipeX,
            y : randomPipeY,
            width : pipeWidth,
            height : pipeHeight,
            passed : false
        }
        pipeArray.push(topPipe);

        let bottomPipe = {
            img : bottomPipeImg,
            x : pipeX,
            y : randomPipeY + pipeHeight + openingSpace,
            width : pipeWidth,
            height : pipeHeight,
            passed : false
        }
        pipeArray.push(bottomPipe);
    }

    function moveBird(e) {
        if (e.code == "Space" || e.code == "ArrowUp" || e.code == "KeyX") {
            //jump
            velocityY = -6;

            //reset game
            if (gameOver) {
                bird.y = birdY;
                pipeArray = [];
                score = 0;
                gameOver = false;
            }
        }
    }

    function detectCollision(a, b) {
        return a.x < b.x + b.width &&   //a's top left corner doesn't reach b's top right corner
            a.x + a.width > b.x &&   //a's top right corner passes b's top left corner
            a.y < b.y + b.height &&  //a's top left corner doesn't reach b's bottom left corner
            a.y + a.height > b.y;    //a's bottom left corner passes b's top left corner
    }
</script> 


<canvas id="board"></canvas>